In [26]:
# Install the gradio_client package silently
!pip install -q gradio_client


In [27]:
import time  # Import the time module for time-related functions
from gradio_client import Client  # Import the Gradio client for interacting with Gradio interfaces
from sklearn.datasets import fetch_openml  # Import fetch_openml to fetch datasets
from sklearn.model_selection import train_test_split  # Import train_test_split for splitting datasets
from tqdm import tqdm  # Import tqdm for progress bars

# Test Data

In [28]:
# Fetch the dataset with ID 42890 in a DataFrame format
dataset = fetch_openml(data_id=42890, as_frame=True, parser="auto")

# Extract the data from the dataset
data_df = dataset.data

# Define the target variable
target = 'Machine failure'

# Define numeric and categorical features
numeric_features = [
    'Air temperature [K]',
    'Process temperature [K]',
    'Rotational speed [rpm]',
    'Torque [Nm]',
    'Tool wear [min]'
]
categorical_features = ['Type']

# Select the features and target variable from the DataFrame
X = data_df[numeric_features + categorical_features]
y = data_df[target]

# Split the dataset into training and testing sets
Xtrain, Xtest, ytrain, ytest = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42
)

In [29]:
# Check the shape of the testing features dataset
Xtest.shape

(2000, 6)

In [30]:
# Display the first 3 rows of the testing features dataset
Xtest.head(3)

,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Type
6252,300.8,310.3,1538,36.1,198,L
4684,303.6,311.8,1421,44.8,101,M
1731,298.3,307.9,1485,42.0,117,M


In [31]:
# Retrieve the values of a specific row (index 6252) from the testing features dataset and convert them to a list
Xtest.loc[6252].tolist()

[300.8, 310.3, 1538, 36.1, 198, 'L']

In [32]:
# Randomly sample 100 rows from the testing features dataset
Xtest_sample = Xtest.sample(100)

In [33]:
# Convert the sampled rows from the DataFrame to a list of tuples
Xtest_sample_rows = list(Xtest_sample.itertuples(index=False, name=None))

In [34]:
# Access the first tuple in the list of sampled rows
Xtest_sample_rows[0]

(300.5, 310.4, 1702, 33.1, 71, 'L')

# Inference

In [35]:
# Create a Gradio client instance for the specified Gradio interface
client = Client("mayankchugh-learning/machinelearning-demo")

Loaded as API: https://mayankchugh-learning-machinelearning-demo.hf.space ✔


## Online/Real time

In [36]:
# Submit a prediction request to the Gradio interface with the given input values
job = client.submit(
    air_temperature=300.5,
    process_temperature=310.5,
    rotational_speed=1695,
    torque=26.7,
    tool_wear=28,
    type="L",
    api_name="/predict"
)

In [37]:
# Print the result of the prediction job
print(job.result())

{'label': '0', 'confidences': None}


## Batch

In [38]:
# Initialize an empty list to store batch predictions
batch_predictions = []

In [39]:
# Iterate through the sampled rows and submit prediction requests to the Gradio interface
# Store the results in batch_predictions list
for row in tqdm(Xtest_sample_rows):
    try:
        # Submit a prediction request for the current row
        job = client.submit(
            air_temperature=row[0],
            process_temperature=row[1],
            rotational_speed=row[2],
            torque=row[3],
            tool_wear=row[4],
            type=row[5],
            api_name="/predict"
        )

        # Append the prediction result to batch_predictions
        batch_predictions.append(job.result())

        # Sleep for 1 second before the next request
        time.sleep(1)

    except Exception as e:
        # Print any exceptions that occur during the prediction
        print(e)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [07:00<00:00,  4.20s/it]


In [40]:
# Display the first 5 predictions from the batch_predictions list
batch_predictions[0:5]

[{'label': '0', 'confidences': None},
 {'label': '0', 'confidences': None},
 {'label': '0', 'confidences': None},
 {'label': '0', 'confidences': None},
 {'label': '0', 'confidences': None}]